# word2vec All Corpora

In [8]:
import sys, os, string, glob, gensim
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize

# Import project-specific functions. 
# Python files (.py) have to be in same folder to work.
lib_path = os.path.abspath(os.path.join(os.path.dirname('JQA_XML_parser.py'), '../Scripts'))
sys.path.append(lib_path)
from JQA_XML_parser import *

# Import project-specific functions. 
# Python files (.py) have to be in same folder to work.
lib_path = os.path.abspath(os.path.join(os.path.dirname('Correspondence_XML_parser.py'), '../Scripts'))
sys.path.append(lib_path)
from Correspondence_XML_parser import *

# Declare absolute path.
abs_dir = "/Users/quinn.wi/Documents/"

## JQA XML Parser

In [11]:
%%time

"""
Declare variables.
"""

# Declare regex to simplify file paths below
regex = re.compile(r'.*/.*/(.*.xml)')

# Declare document level of file. Requires root starting point ('.').
doc_as_xpath = './/ns:div/[@type="entry"]'

# Declare text level within each document.
text_path = './ns:div/[@type="docbody"]/ns:p'

"""
Build dataframe.
"""

dataframe = []

for file in glob.glob(abs_dir + 'Data/PSC/JQA/*/*.xml'):
    reFile = str(regex.search(file).group(1))
#         Call functions to create necessary variables and grab content.
    root = get_root(file)
    ns = get_namespace(root)

    for eachDoc in root.findall(doc_as_xpath, ns):
#             Call functions.
        text = get_textContent(eachDoc, text_path, ns)

        dataframe.append([text])

dataframe = pd.DataFrame(dataframe, columns = ['text'])

CPU times: user 4.61 s, sys: 102 ms, total: 4.71 s
Wall time: 4.77 s


## Gather Files

In [12]:
%%time

# Richards
# Declare directory location to shorten filepaths later.
input_directory = "Data/PSC/Richards/ESR-XML-Files-MHS/*.xml"
files = glob.glob(abs_dir + input_directory)

# Sedgwick
input_directory = "Data/PSC/Sedgwick/*.xml"
files = files + glob.glob(abs_dir + input_directory)

# Taney
input_directory = "Data/PSC/Taney/RBT_RawXML/*/*.xml"
files = files + glob.glob(abs_dir + input_directory)

df = build_dataframe(files)
df = df[['text']]

dataframe = pd.concat([dataframe, df], ignore_index=True)

/Users/quinn.wi/Documents/Data/PSC/Richards/ESR-XML-Files-MHS/ESR-EDA-1893-09-24.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1803-10-06-toPamelaDwightSedgwickF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1809-01-27-toTheodoreSedgwickIFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-25-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1806-01-17-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-29-toPamelaDwightSedgwickFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFSWF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1800-01-12-toTheodoreSedgwickIF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-15-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-28-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sed

## Build Model

In [13]:
# Define tokenizer.
def fast_tokenize(text):
    
    # Get a list of punctuation marks
    punct = string.punctuation + '“' + '”' + '‘' + "’"
    
    lower_case = text.lower()
    lower_case = lower_case.replace('—', ' ').replace('\n', ' ')
    
    # Iterate through text removing punctuation characters
    no_punct = "".join([char for char in lower_case if char not in punct])
    
    # Split text over whitespace into list of words
    tokens = no_punct.split()
    
    return tokens

In [15]:
%%time

# Convert dataframe text field to list of sentences.
sentences = [sentence for text in dataframe['text'] for sentence in sent_tokenize(text)]
words_by_sentence = [fast_tokenize(sentence) for sentence in sentences]
words_by_sentence = [sentence for sentence in words_by_sentence if sentence != []]

# Get total number of words and unique words.
single_list_of_words = []
for l in words_by_sentence:
    for w in l:
        single_list_of_words.append(w)
print (f'Word total: {len(single_list_of_words)}\nUnique word total {len(set(single_list_of_words))}')

# Build model.
model = gensim.models.Word2Vec(words_by_sentence, window=10, vector_size=300,
                               min_count=10, sg=1, alpha=0.025, batch_words=10000, workers=4)

# Unused arguments:
# size=100, iter=5,

# Save model for later use
model.wv.save_word2vec_format(abs_dir + '/Data/Output/WordVectors/w2v_allCorpora.txt')

Word total: 6442033
Unique word total 62201
CPU times: user 6min 7s, sys: 1.93 s, total: 6min 9s
Wall time: 1min 45s
